# Nutrition Agent Demo 🥗🤖

This notebook demonstrates the Nutrition Agent capabilities with interactive examples and evaluation results.

## Setup and Imports

In [ ]:
import sys

sys.path.append("..")

import json
from pathlib import Path

# Jupyter async support
import nest_asyncio

from agents.base_agent import AgentConfig, ModelProvider
from agents.nutrition_planner import (
    DietaryConstraints,
    Inventory,
    NutritionPlannerAgent,
)
from evaluators.nutrition_evaluator import NutritionEvaluator

nest_asyncio.apply()

## Example 1: Basic Meal Planning

Let's create a simple meal plan with basic ingredients.

In [ ]:
# Define available ingredients
inventory = Inventory(
    items=[
        {"name": "chicken_breast", "amount_g": 400, "unit": "g"},
        {"name": "eggs", "amount_g": 360, "unit": "g"},
        {"name": "milk", "amount_g": 500, "unit": "ml"},
        {"name": "white_rice", "amount_g": 300, "unit": "g"},
        {"name": "lettuce", "amount_g": 200, "unit": "g"},
    ]
)

# Define dietary constraints
constraints = DietaryConstraints(
    daily_calories=2000.0,
    pfc_ratio=(30.0, 25.0, 45.0),  # 30% protein, 25% fat, 45% carbs
    allergens=None,
    dietary_restrictions=None,
)

print("🥘 Inventory:")
for item in inventory.items:
    print(f"  • {item['name']}: {item['amount_g']}{item['unit']}")

print("\n🎯 Targets:")
print(f"  • Daily calories: {constraints.daily_calories} kcal")
print(
    f"  • PFC ratio: {constraints.pfc_ratio[0]}% / {constraints.pfc_ratio[1]}% / {constraints.pfc_ratio[2]}%"
)

In [ ]:
# Create agent with GPT-4o
config = AgentConfig(
    model_provider=ModelProvider.OPENAI, model_name="gpt-4o", temperature=0.7
)

agent = NutritionPlannerAgent(config)

# Generate meal plan
print("🤖 Generating 3-day meal plan...")
meal_plans = await agent.generate_meal_plan(inventory, constraints, days=3)

# Display results
agent.display_meal_plans(meal_plans)

In [ ]:
# Generate shopping list
shopping_list = agent.generate_shopping_list(meal_plans)
agent.display_shopping_list(shopping_list)

## Example 2: Vegetarian Meal Planning

Let's try a vegetarian scenario with plant-based proteins.

In [ ]:
# Load vegetarian test scenario
with open("../data/test_prompts/t2.json") as f:
    vegetarian_scenario = json.load(f)

veg_inventory = Inventory(items=vegetarian_scenario["inventory"])
veg_constraints = DietaryConstraints(**vegetarian_scenario["constraints"])

print("🌱 Vegetarian Scenario:")
print(f"Description: {vegetarian_scenario['description']}")
print(f"Notes: {vegetarian_scenario['notes']}")

# Generate vegetarian meal plan
veg_meal_plans = await agent.generate_meal_plan(veg_inventory, veg_constraints, days=3)
agent.display_meal_plans(veg_meal_plans[:1])  # Show first day only for brevity

## Example 3: Model Comparison

Compare performance across different models.

In [ ]:
# Test different models
models_to_test = [
    ("GPT-4o", AgentConfig(ModelProvider.OPENAI, "gpt-4o", 0.7)),
    ("GPT-3.5-turbo", AgentConfig(ModelProvider.OPENAI, "gpt-3.5-turbo", 0.7)),
]

comparison_results = []

for model_name, model_config in models_to_test:
    print(f"\n🔍 Testing {model_name}...")

    try:
        test_agent = NutritionPlannerAgent(model_config)
        test_plans = await test_agent.generate_meal_plan(inventory, constraints, days=1)

        # Simple performance metrics
        daily_nutrition = test_plans[0].daily_nutrition
        calories_error = (
            abs(daily_nutrition["total_calories"] - constraints.daily_calories)
            / constraints.daily_calories
            * 100
        )

        comparison_results.append(
            {
                "model": model_name,
                "calories": daily_nutrition["total_calories"],
                "calories_error": calories_error,
                "missing_ingredients": len(test_plans[0].missing_ingredients),
            }
        )

        print(
            f"  ✅ Calories: {daily_nutrition['total_calories']:.0f} (error: {calories_error:.1f}%)"
        )
        print(f"  📝 Missing ingredients: {len(test_plans[0].missing_ingredients)}")

    except Exception as e:
        print(f"  ❌ Error: {e}")
        comparison_results.append({"model": model_name, "error": str(e)})

## Example 4: Evaluation System Demo

Demonstrate the automated evaluation system.

In [ ]:
# Create evaluator
evaluator = NutritionEvaluator()

# Load test scenario
scenario_path = Path("../data/test_prompts/t1.json")

# Run evaluation
print("📊 Running evaluation on T1 scenario...")
evaluation_result = await evaluator.evaluate_scenario(scenario_path, config, days=3)

print("\n📈 Evaluation Results:")
print(f"Overall Score: {evaluation_result.score:.3f}/1.0")
print(f"Nutrition Score: {evaluation_result.nutrition_score:.3f}/0.5")
print(f"Shopping List Score: {evaluation_result.shopping_list_score:.3f}/0.5")
print(f"Jaccard Similarity: {evaluation_result.jaccard_similarity:.3f}")
print(f"Execution Time: {evaluation_result.execution_time:.1f}s")

if evaluation_result.violations:
    print("\n⚠️ Violations:")
    for violation in evaluation_result.violations:
        print(f"  • {violation}")

if evaluation_result.nutrition_errors:
    print("\n🔍 Nutrition Errors:")
    for macro, error in evaluation_result.nutrition_errors.items():
        print(f"  • {macro}: {error:.1f}%")

## Example 5: Advanced Features Demo

In [ ]:
# Demo with dietary restrictions and allergies
advanced_constraints = DietaryConstraints(
    daily_calories=1800.0,
    pfc_ratio=(25.0, 30.0, 45.0),
    allergens=["dairy", "nuts"],
    dietary_restrictions=["vegetarian", "low-sodium"],
)

print("🚫 Advanced constraints:")
print(f"  • Allergens: {', '.join(advanced_constraints.allergens)}")
print(f"  • Restrictions: {', '.join(advanced_constraints.dietary_restrictions)}")
print(f"  • Lower calories: {advanced_constraints.daily_calories} kcal")
print(f"  • Higher fat ratio: {advanced_constraints.pfc_ratio[1]}%")

# Note: This would require actual LLM calls, so we'll simulate the expected behavior
print("\n💭 Expected agent behavior:")
print("  • Avoid dairy products (milk, cheese, butter)")
print("  • Avoid nuts and nut-based products")
print("  • Focus on plant-based proteins (tofu, legumes)")
print("  • Use herbs/spices instead of salt for flavoring")
print("  • Adjust portions for lower calorie target")

## Performance Analysis

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Sample performance data (would come from actual evaluation runs)
models = ["GPT-4o", "GPT-3.5-turbo", "Claude-3-Sonnet"]
nutrition_scores = [0.432, 0.378, 0.401]
shopping_scores = [0.415, 0.345, 0.390]
execution_times = [23.2, 8.7, 18.5]

# Create comparison charts
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8))

# Score comparison
x = np.arange(len(models))
width = 0.35

ax1.bar(x - width / 2, nutrition_scores, width, label="Nutrition Score", alpha=0.8)
ax1.bar(x + width / 2, shopping_scores, width, label="Shopping Score", alpha=0.8)
ax1.set_xlabel("Models")
ax1.set_ylabel("Score")
ax1.set_title("Model Performance Comparison")
ax1.set_xticks(x)
ax1.set_xticklabels(models)
ax1.legend()
ax1.set_ylim(0, 0.5)

# Overall scores
overall_scores = [
    n + s for n, s in zip(nutrition_scores, shopping_scores, strict=False)
]
ax2.bar(models, overall_scores, color=["#1f77b4", "#ff7f0e", "#2ca02c"], alpha=0.8)
ax2.set_ylabel("Overall Score")
ax2.set_title("Overall Performance")
ax2.set_ylim(0, 1.0)

# Execution time
ax3.bar(models, execution_times, color=["#d62728", "#9467bd", "#8c564b"], alpha=0.8)
ax3.set_ylabel("Time (seconds)")
ax3.set_title("Execution Time")

# Cost-performance trade-off (hypothetical data)
costs = [0.12, 0.03, 0.08]  # USD per run
ax4.scatter(costs, overall_scores, s=100, alpha=0.7)
for i, model in enumerate(models):
    ax4.annotate(
        model, (costs[i], overall_scores[i]), xytext=(5, 5), textcoords="offset points"
    )
ax4.set_xlabel("Cost per run ($)")
ax4.set_ylabel("Overall Score")
ax4.set_title("Cost vs Performance")

plt.tight_layout()
plt.show()

# Print summary
print("📊 Performance Summary:")
for i, model in enumerate(models):
    print(f"  {model}:")
    print(f"    Overall Score: {overall_scores[i]:.3f}")
    print(f"    Execution Time: {execution_times[i]}s")
    print(f"    Cost Efficiency: {overall_scores[i] / costs[i]:.1f} score/$")

## Key Learnings and Next Steps

### What Worked Well:
1. **Tool-using agent pattern**: Clean separation of concerns with FatSecret API integration
2. **Structured evaluation**: Automated scoring system enables objective model comparison
3. **Rich CLI interface**: User-friendly interaction with comprehensive output formatting
4. **Flexible architecture**: Easy to extend with new dietary constraints and evaluation metrics

### Challenges Encountered:
1. **API rate limiting**: Required careful request management and caching
2. **Nutrition data consistency**: Different food databases have varying nutritional information
3. **Response parsing**: LLM outputs needed robust validation and error handling
4. **Evaluation complexity**: Balancing multiple objectives (nutrition, cost, variety) is non-trivial

### Future Improvements:
- **Multi-agent system**: Separate agents for planning, validation, and optimization
- **Learning from feedback**: Adapt to user preferences over time
- **Recipe generation**: Create novel recipes instead of just searching existing ones
- **Seasonal awareness**: Factor in ingredient availability and seasonality
- **Batch cooking optimization**: Consider meal prep and leftover management